In [14]:
import sqlite3
from sqlite3 import Error
import pandas as pd
database = "Data_Multi_Table.db"

In [15]:
def create_connection(db_file):
    """ create a database connection to the SQLite database
        specified by db_file
    :param db_file: database file
    :return: Connection object or None
    """
    conn = None
    try:
        conn = sqlite3.connect(db_file)
        return conn
    except Error as e:
        print(e)
 
    return conn

In [16]:
conn = create_connection(database)

In [17]:
file = r'Prediction_inflation_adjusted.csv'
clean_data = pd.read_csv(file, encoding='utf-8')
columns = clean_data.columns
c = conn.cursor()

In [18]:
clean_data.to_sql('PREDICTION', conn, if_exists='append', index = False) # Insert the values from the csv file into the table 

In [19]:
clean_data['YEAR']= clean_data['YEAR'].astype(str)
clean_data['PRIMARY_KEY'] = clean_data['STATE'] +"_"+ clean_data["YEAR"]
clean_data.head()


,STATE,YEAR,Inflation_pegged_2019,TOTAL_REVENUE,FEDERAL_REVENUE,STATE_REVENUE,LOCAL_REVENUE,TOTAL_EXPENDITURE,INSTRUCTION_EXPENDITURE,SUPPORT_SERVICES_EXPENDITURE,OTHER_EXPENDITURE,CAPITAL_OUTLAY_EXPENDITURE,MEDIAN_INCOME,ALL_STUDENT_TEACHER_RATIO,AVG_ALL_SCORES,ALL_STUDENTS,PRIMARY_KEY
0,ALABAMA,2003,0.395,15.796304,13.582250,15.235950,14.656047,15.815910,15.184117,14.568077,13.291943,13.309721,10.839346,15.692794,238.559596,13.513548,ALABAMA_2003
1,ALASKA,2003,0.395,14.503242,12.799110,13.941837,13.107554,14.624819,13.878596,13.482974,11.260899,12.750058,11.169660,16.629208,248.084075,11.808308,ALASKA_2003
2,ARIZONA,2003,0.395,16.024796,13.848082,15.217461,15.205044,15.974612,15.181912,14.823505,12.957427,14.002965,10.939160,19.909450,244.192629,13.751244,ARIZONA_2003
3,ARKANSAS,2003,0.395,15.324372,13.180681,15.021511,13.386962,15.324842,14.718657,14.120621,12.380950,12.751029,10.687343,14.869271,243.262509,13.019189,ARKANSAS_2003
4,CALIFORNIA,2003,0.395,18.239676,15.905513,17.663460,17.164059,18.238572,17.534880,16.983401,15.245605,16.141954,11.119483,20.644608,238.802575,15.664543,CALIFORNIA_2003


In [20]:
cols = list(clean_data.columns)
cols = [cols[-1]] + cols[:-1]
clean_data =clean_data[cols]
clean_data.head()

,PRIMARY_KEY,STATE,YEAR,Inflation_pegged_2019,TOTAL_REVENUE,FEDERAL_REVENUE,STATE_REVENUE,LOCAL_REVENUE,TOTAL_EXPENDITURE,INSTRUCTION_EXPENDITURE,SUPPORT_SERVICES_EXPENDITURE,OTHER_EXPENDITURE,CAPITAL_OUTLAY_EXPENDITURE,MEDIAN_INCOME,ALL_STUDENT_TEACHER_RATIO,AVG_ALL_SCORES,ALL_STUDENTS
0,ALABAMA_2003,ALABAMA,2003,0.395,15.796304,13.582250,15.235950,14.656047,15.815910,15.184117,14.568077,13.291943,13.309721,10.839346,15.692794,238.559596,13.513548
1,ALASKA_2003,ALASKA,2003,0.395,14.503242,12.799110,13.941837,13.107554,14.624819,13.878596,13.482974,11.260899,12.750058,11.169660,16.629208,248.084075,11.808308
2,ARIZONA_2003,ARIZONA,2003,0.395,16.024796,13.848082,15.217461,15.205044,15.974612,15.181912,14.823505,12.957427,14.002965,10.939160,19.909450,244.192629,13.751244
3,ARKANSAS_2003,ARKANSAS,2003,0.395,15.324372,13.180681,15.021511,13.386962,15.324842,14.718657,14.120621,12.380950,12.751029,10.687343,14.869271,243.262509,13.019189
4,CALIFORNIA_2003,CALIFORNIA,2003,0.395,18.239676,15.905513,17.663460,17.164059,18.238572,17.534880,16.983401,15.245605,16.141954,11.119483,20.644608,238.802575,15.664543


In [21]:
clean_data.to_sql('PREDICTIONS', conn, if_exists='append', index = False) # Insert the values from the csv file into the table 

In [22]:
#Create financial table
c.execute('''CREATE TABLE IF NOT EXISTS FINANCIAL ([PRIMARY_KEY] text PRIMARY KEY, [STATE] text, [YEAR] date,[TOTAL_REVENUE] REAL, [FEDERAL_REVENUE] REAL, 
                [STATE_REVENUE] REAL, [LOCAL_REVENUE] REAL,[TOTAL_EXPENDITURE] REAL, [INSTRUCTION_EXPENDITURE] REAL
                ,[SUPPORT_SERVICES_EXPENDITURE],[OTHER_EXPENDITURE] REAL, [CAPITAL_OUTLAY_EXPENDITURE] REAL)''')
 

                 
conn.commit()

In [23]:
#Create income table
c.execute('''CREATE TABLE IF NOT EXISTS INCOME ([PRIMARY_KEY] text PRIMARY KEY, [STATE] text, [YEAR] date, [MEDIAN_INCOME])''')
conn.commit()

In [24]:
#Create scores_ratios table
c.execute('''CREATE TABLE IF NOT EXISTS SCORES_RATIOS ([PRIMARY_KEY] text PRIMARY KEY, [STATE] text, [YEAR] date, [ALL_STUDENT_TEACHER_RATIO],
[AVG_ALL_SCORES],[ALL_STUDENTS])''')         
conn.commit()

In [25]:
#Pushing Data into FINANCIAL
c.execute('''
INSERT INTO FINANCIAL ('PRIMARY_KEY', 'STATE', 'YEAR',  'TOTAL_REVENUE',
       'FEDERAL_REVENUE', 'STATE_REVENUE', 'LOCAL_REVENUE',
       'TOTAL_EXPENDITURE', 'INSTRUCTION_EXPENDITURE',
       'SUPPORT_SERVICES_EXPENDITURE', 'OTHER_EXPENDITURE',
       'CAPITAL_OUTLAY_EXPENDITURE') 
         SELECT DISTINCT p.PRIMARY_KEY, p.STATE, p.YEAR, p.TOTAL_REVENUE,
        p.FEDERAL_REVENUE, p.STATE_REVENUE, p.LOCAL_REVENUE,
        p.TOTAL_EXPENDITURE, p.INSTRUCTION_EXPENDITURE,
        p.SUPPORT_SERVICES_EXPENDITURE, p.OTHER_EXPENDITURE, p.CAPITAL_OUTLAY_EXPENDITURE FROM PREDICTIONS p''')
conn.commit()

In [26]:
#Pushing Data into INCOME
c.execute('''INSERT INTO INCOME ('PRIMARY_KEY', 'STATE', 'YEAR', 'MEDIAN_INCOME') SELECT DISTINCT p.PRIMARY_KEY, p.STATE, p.YEAR,P.MEDIAN_INCOME FROM PREDICTIONS P ''')
conn.commit()

In [27]:
#Pushing Data into SCORES_RATIOS
c.execute('''INSERT INTO SCORES_RATIOS ('PRIMARY_KEY', 'STATE', 'YEAR', 'ALL_STUDENT_TEACHER_RATIO','AVG_ALL_SCORES','ALL_STUDENTS') SELECT DISTINCT p.PRIMARY_KEY, p.STATE, 
p.YEAR, P.ALL_STUDENT_TEACHER_RATIO, P.AVG_ALL_SCORES,  P.ALL_STUDENTS FROM PREDICTIONS P ''')
conn.commit()

In [28]:
#View of all data
df = pd.read_sql_query('SELECT * FROM SCORES_RATIOS S INNER JOIN INCOME I ON I.PRIMARY_KEY = S.PRIMARY_KEY INNER JOIN FINANCIAL F ON F.PRIMARY_KEY = S.PRIMARY_KEY;', conn)
df.head()

,PRIMARY_KEY,STATE,YEAR,ALL_STUDENT_TEACHER_RATIO,AVG_ALL_SCORES,ALL_STUDENTS,PRIMARY_KEY,STATE,YEAR,MEDIAN_INCOME,...,YEAR,TOTAL_REVENUE,FEDERAL_REVENUE,STATE_REVENUE,LOCAL_REVENUE,TOTAL_EXPENDITURE,INSTRUCTION_EXPENDITURE,SUPPORT_SERVICES_EXPENDITURE,OTHER_EXPENDITURE,CAPITAL_OUTLAY_EXPENDITURE
0,ALABAMA_2003,ALABAMA,2003,15.692794,238.559596,13.513548,ALABAMA_2003,ALABAMA,2003,10.839346,...,2003,15.796304,13.582250,15.235950,14.656047,15.815910,15.184117,14.568077,13.291943,13.309721
1,ALASKA_2003,ALASKA,2003,16.629208,248.084075,11.808308,ALASKA_2003,ALASKA,2003,11.169660,...,2003,14.503242,12.799110,13.941837,13.107554,14.624819,13.878596,13.482974,11.260899,12.750058
2,ARIZONA_2003,ARIZONA,2003,19.909450,244.192629,13.751244,ARIZONA_2003,ARIZONA,2003,10.939160,...,2003,16.024796,13.848082,15.217461,15.205044,15.974612,15.181912,14.823505,12.957427,14.002965
3,ARKANSAS_2003,ARKANSAS,2003,14.869271,243.262509,13.019189,ARKANSAS_2003,ARKANSAS,2003,10.687343,...,2003,15.324372,13.180681,15.021511,13.386962,15.324842,14.718657,14.120621,12.380950,12.751029
4,CALIFORNIA_2003,CALIFORNIA,2003,20.644608,238.802575,15.664543,CALIFORNIA_2003,CALIFORNIA,2003,11.119483,...,2003,18.239676,15.905513,17.663460,17.164059,18.238572,17.534880,16.983401,15.245605,16.141954


In [29]:
#Removing Uneccesary Column
c.execute('''DROP TABLE predictions''')
conn.commit()